In [ ]:
import os
import shutil
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
from copy import deepcopy

from pyproj import Transformer, CRS
set_crs_20254 = "epsg:20254"
epsgstring = "EPSG:28354"
suffix = epsgstring.split(':')[-1]

transformer = Transformer.from_crs(golemepsgstring, set_crs_20254)
transformer = Transformer.from_crs(set_crs_20254, golemepsgstring)

## Changing DXF Coordinates

Basically, taking a DXF as text file approach.

Parse it, find coordinate sets, reproject the X and Y, rewrite file.

This is fine if not too many of them, if they have millions of coordinates, will want to do a vectorised/parallelised version.

## Details

As with any parsing problem, you need to find the areas of interest and how to demarcate them.

In the example I was using, you have AcDbFace and 3DFACE for example.  If in a parsing job you see ENDSEC in a file - good chance that sort of notation will be helpful, too.

Read the file into a dataframe, take advantage of the index to build a dictionary of the coordinate relevant parts.

Make a new dictionary with updated coordinates, use that to replace the lines in the original file and write out.

In [ ]:
basedir = r'projectdir'
outdir = r'output'

donelist = []  #if multiple runs with some big ones finished already, etc.

for root, dirs, files in os.walk(basedir):
    for file in files:
        print("processing: ", file)
        if 'dxf' in file and file not in donelist:
            with open(os.path.join(root,file)) as f:
                lines = f.readlines()            
                df = pd.DataFrame()
                df['dxf'] = lines
                acdb = df.loc[df.dxf.str.contains('AcDbFace')]
                threed = df.loc[df.dxf.str.contains('3DFACE')]
                
                acdbindex = acdb.index.tolist()
                threedindex = threed.index.tolist()                
                print(df.shape, acdb.shape, threed.shape)  #check this is making sense
                
                flag = False
                polygondict = {}
                for index, row in tqdm(df.iterrows(), total = df.shape[0]):
                    if index in acdbindex:
                        acdbidx = index
                        flag = True
                        polygondict[acdbidx] = []
                    if index in threedindex or 'ENDSEC' in row['dxf']:
                        flag = False
                    if flag == True:
                        polygondict[acdbidx].append(row['dxf'])
                        
                polygondict_new = deepcopy(polygondict)

                for key in polygondict:
                    if 'AcDbFace\n' not in polygondict[key]:
                        continue
                        
                    coords_len = len(polygondict[key]) // 6
                    for c in range(coords_len ):
                        x = polygondict[key][c*6 + 2]
                        y = polygondict[key][c*6 + 4]
                        z = polygondict[key][c*6 + 6]
                        xnum = x.strip()
                        ynum = y.strip()
                        znum = z.strip()
                        origin28354 = transformer.transform(xnum, ynum)

                        polygondict_new[key][c*6 + 2] = str(origin28354[0]) + "\n"
                        polygondict_new[key][c*6 + 4] = str(origin28354[1]) + "\n"
                        
                linesnew = deepcopy(lines)
                for key in polygondict_new:
                    for i, pdata in enumerate(polygondict_new[key]):
                        linesnew[key + i] = polygondict_new[key][i]                        
                    
                newfile = file.replace('.dxf','_' + suffix + '.dxf')
                with open(outdir + newfile,'w') as f:
                    for l in linesnew:
                        f.write(l)                    
